In [73]:
import pandas as pd
import numpy as np
import panel as pn
import holoviews as hv
from holoviews import opts
import hvplot.pandas
pn.extension()
hv.extension('bokeh')

In [56]:
df = pd.read_csv('Derived/All-Journals-Cleaned.csv')

In [57]:
# Data Manipulations for Later Use
# create a year column
df["Year"] = (df['Issue'].str[-4:]).astype(int)

# identify author and JEL columns
author_columns = [col for col in df.columns if col.startswith("Author")]
jel_columns = [col for col in df.columns if col.startswith("JEL")]

## Widgets Creation

In [85]:
# Define Panel Widgets

# 1. Journal Dropdown
journal_names = df['Journal'].unique()
journal_names = list(journal_names)
journal_select = pn.widgets.Select(name='Choose Journal', options=journal_names)

# 2. Year Slider
journal_start_yr = {
    'AMERICAN ECONOMIC REVIEW' : 1999,
    'AMERICAN ECONOMIC REVIEW: INSIGHTS' : 2019,
    'AMERICAN ECONOMIC JOURNAL: APPLIED ECONOMICS' : 2009,
    'AMERICAN ECONOMIC JOURNAL: ECONOMIC POLICY' : 2009,
    'AMERICAN ECONOMIC JOURNAL: MACROECONOMICS' : 2009,
    'AMERICAN ECONOMIC JOURNAL: MICROECONOMICS' : 2009
}
    
year_slider = pn.widgets.IntSlider(name = "Start Year", start = 1999, end = 2024, step = 1)

# Function to update the year slider range based on the journal selected
def update_year_range(event):
    journal_name = event.new
    year_slider.start = journal_start_yr[journal_name]
    year_slider.value = journal_start_yr[journal_name]

# Watch for changes to the choice of journal
journal_select.param.watch(update_year_range, 'value')

Watcher(inst=Select(name='Choose Journal', options=['AMERICAN ECONOMIC REVIEW...], value='AMERICAN ECONOMIC R...), cls=<class 'panel.widgets.select.Select'>, fn=<function update_year_range at 0x7f81a8ccfdc0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

## Top Economists Panel

In [59]:
# Top Economists
def get_top_authors(start_year, journal):
    """
    function to get top 10 authors in the selected journal from the selected start year
    """
    # keep paper in the selected journal from the selected start year
    filtered_data = df[(df['Year'] >= start_year) & (df['Journal'] == journal)]
    # transform the dataset so that each row corresponds to a single author of a paper
    filtered_melted_data = filtered_data.melt(id_vars='Title', value_vars=author_columns, var_name='AuthorCol', value_name='Author').dropna(subset=['Author'])
    # get the top 10 authors with most works published
    author_counts_top10 = filtered_melted_data['Author'].value_counts().head(10)
    return author_counts_top10.to_frame('Number of Papers')

## Research Field Trend Panel

In [93]:
# Popular Research Fields
# code book for subfields
field_code = {
    "A" : "General Economics and Teaching",
    "B" : "History of Economic Thought, Methodology, and Heterodox Approaches",
    "C" : "Mathematical and Quantitative Methods",
    "D" : "Microeconomics",
    "E" : "Macroeconomics and Monetary Economics",
    "F" : "International Economics",
    "G" : "Financial Economics",
    "H" : "Public Economics",
    "I" : "Health, Education, and Welfare",
    "J" : "Labor and Demographic Economics",
    "K" : "Law and Economics",
    "L" : "Industrial Organization",
    "M" : "Business Administration and Business Economics • Marketing • Accounting • Personnel Economics",
    "N" : "Economic History",
    "O" : "Economic Development, Innovation, Technological Change, and Growth",
    "P" : "Political Economy and Comparative Economic Systems",
    "Q" : "Agricultural and Natural Resource Economics • Environmental and Ecological Economics",
    "R" : "Urban, Rural, Regional, Real Estate, and Transportation Economics",
    "Y" : "Miscellaneous Categories",
    "Z" : "Other Special Topics"
}

# construct a sub dataset which contains annual counts of works in each subfiled of a journal
    # transform the dataset so that each row corresponds to a single JEL code of a paper
melted_data_by_JEL = df.melt(id_vars=['Title','Year','Journal'], value_vars=jel_columns, var_name='JELCol', value_name='JEL').dropna(subset=['JEL'])
    # create a column of corresponding subfields
melted_data_by_JEL['Subfield'] = melted_data_by_JEL['JEL'].str[0].map(field_code)
    # group by year and subfield, then count the occurrences
grouped_subfield = melted_data_by_JEL.groupby(['Year','Journal', 'Subfield']).size().reset_index(name='counts')
    # rename columns for clarity
grouped_subfield.columns = ['Year', 'Journal', 'Subfield', 'Count']
    # make dataframe pipline interactive
igrouped_subfield = grouped_subfield.interactive()

In [95]:
# create subfiled works count pipline
subfiled_count_pipline = (
    igrouped_subfield[
    (igrouped_subfield.Year >= year_slider) &
    (igrouped_subfield.Journal == journal_select)
    ].reset_index(drop=True)
)

In [106]:
# subfield trend plot
subfiled_trend_plot = subfiled_count_pipline.hvplot(x='Year', y='Count', by='Subfield', title='Research Field Trend',height=400, width=1200)
subfiled_trend_plot

In [75]:
# Reactive function to update the table and plots based on selectors
@pn.depends(year_slider, journal_select)
def update_dashboard(start_year, journal):
    top_economists_table = get_top_authors(start_year, journal)
    subfield_plot = subfiled_trend_plot(start_year,journal)
    return pn.Column(subfield_plot, top_economists_table)



In [76]:
# Layout
dashboard = pn.Column(
    pn.Row(year_slider, journal_select),
    update_dashboard
)

dashboard.show()

Launching server at http://localhost:63263


ERROR:tornado.application:Uncaught exception GET / (::1)
HTTPServerRequest(protocol='http', host='localhost:63263', method='GET', uri='/', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/Users/yirui/Desktop/Global_Affair_HW/final-project-aea/lib/python3.9/site-packages/tornado/web.py", line 1790, in _execute
    result = await result
  File "/Users/yirui/Desktop/Global_Affair_HW/final-project-aea/lib/python3.9/site-packages/panel/io/server.py", line 588, in get
    session = await self.get_session()
  File "/Users/yirui/Desktop/Global_Affair_HW/final-project-aea/lib/python3.9/site-packages/panel/io/server.py", line 477, in get_session
    session = await super().get_session()
  File "/Users/yirui/Desktop/Global_Affair_HW/final-project-aea/lib/python3.9/site-packages/bokeh/server/views/session_handler.py", line 145, in get_session
    session = await self.application_context.create_session_if_needed(session_id, self.request, token)
  File "/Users/yirui/D